# Introduction to quantum gates with QISKit

This notebook introduces quantum logic gates in QISKit. After working through this notebook a user will have knowledge about quantum logic gates, how they operate on qubits, and how they are implemented in QISKit. Quantum logic gates (along with qubits) form all the needed components to create a quantum circuit.

## Quantum logic gates
Recall that a single qubit is represented as a 2-D column vector. Operations on a single qubit are achieved by  multiplying the vector with a $2 \times 2$ matrix. If we have a 2-D column vector $x = \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$ and a $ 2 \times 2$ matrix $A = \begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22}\end{bmatrix}$, we can multiply them together as follows:

$$
Ax = 
\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22}\end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
= 
\begin{bmatrix} a_{11}x_1 + a_{12}x_2\\ a_{21}x_1 + a_{22}x_2 \end{bmatrix}.
$$

A high-level perspective of $2\times 2$ matrix-vector multiplication is that it is a transforms one vector to another vector. The transformation details are encoded in the matrix. 